# Manage biological registries 

The standalone package [Bionty](https://lamin.ai/docs/bionty/) helps with curating metadata using ontologies.

If you only work with pre-defined ontologies (public or in-house), Bionty is sufficient - check out its [docs](https://lamin.ai/docs/bionty/)!

If you'd like to maintain in-house registries along with ontologies, we recommend managing them using `lnschema_bionty`'s ORMs.

In [ ]:
!lamin delete test-ontologies
!lamin init --storage ./test-ontologies --schema bionty

In [ ]:
import lamindb as ln
import lnschema_bionty as lb

## Lookup a public ontology entry

In [ ]:
celltype_bionty = lb.CellType.bionty()  # same as bionty.CellType()
celltype_bionty_lookup = celltype_bionty.lookup()

In [ ]:
len(celltype_bionty_lookup)

In [ ]:
celltype_bionty_lookup.gamma_delta_T_cell

## Create a record for an in-house ontology

In [ ]:
celltype_record = lb.CellType.from_bionty(celltype_bionty_lookup.gamma_delta_T_cell)

celltype_record

(Note: The id here is a hash of the ontology entry.)

You can add it to the DB to seed an in-house ontology:

In [ ]:
ln.save(celltype_record)

In [ ]:
ln.select(lb.CellType).df()

In [ ]:
ln.select(lb.CellType, name=celltype_record.name).one()

You can now work with a lookup object with much less terms: `lb.CellType.lookup()`

## Parse records from data

Often, you want to parse records from data and map it onto a reference. {func}`~lamindb.parse` takes any iterable and maps it on your in-house reference.

Consider a DataFrame-based example:

In [ ]:
adata = ln.dev.datasets.anndata_with_obs()

In [ ]:
adata.obs.head()

In [ ]:
adata.obs.cell_type.value_counts()

You can parse the cell types and create records in 3 ways:

1. parse based on cell type name column
2. parse based on cell type id column
3. parse based on both columns

Use the cell type name column:

In [ ]:
cell_types = ln.parse(adata.obs.cell_type, lb.CellType.name)

cell_types

Use the cell type id column, which has an empty string for "my new cell type":

In [ ]:
ln.parse(adata.obs.cell_type_id, lb.CellType.ontology_id)

Use both columns:

In [ ]:
ln.parse(
    adata.obs,
    {"cell_type_id": lb.CellType.ontology_id, "cell_type": lb.CellType.name},
)

(Note: no additional fields are mapped from bionty if multiple columns are parsed.)

If we're happy with `cell_types`, we commit them to the DB:

In [ ]:
ln.save(cell_types);

Our in-house registry grew a bit:

In [ ]:
ln.select(lb.CellType).df()

This also works for any other entity in Bionty.

## Automated tracking of underlying ontology sources

Under-the-hood, ontology sources are tracked:

In [ ]:
ln.select(lb.BiontySource, currently_used=True).df()

Each record is linked to a bionty source if they are created from bionty:

In [ ]:
cell_type_record = ln.select(lb.CellType, name="hepatocyte").one()
cell_type_record.bionty_source

In [ ]:
!lamin delete test-ontologies
!rm -r test-ontologies